# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
Arxiv has 66 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2312.02311


extracting tarball to tmp_2312.02311...

 done.
Retrieving document from  https://arxiv.org/e-print/2312.02653


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2312.02311/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

✘ → 10946:\section{Setup}
  ↳ 20369:\section{Results}


✔ → 20369:\section{Results}
  ↳ 37326:\section{Variation of the orbital elements}
✔ → 37326:\section{Variation of the orbital elements}
  ↳ 38654:\section{Initial conditions and Kepler equation}


✔ → 38654:\section{Initial conditions and Kepler equation}
  ↳ 40101:\section{Coordinate transformation}
✔ → 40101:\section{Coordinate transformation}
  ↳ 41545:\section{Relativistic effects and R{\O}mer's delay}


✔ → 41545:\section{Relativistic effects and R{\O}mer's delay}
  ↳ 43601:\section{MCMC details}
✔ → 43601:\section{MCMC details}
  ↳ 46893:end


Found 63 bibliographic references in tmp_2312.02653/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.02653-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.02653) | **Using the motion of S2 to constrain vector clouds around SgrA***  |
|| G. Collaboration, et al. -- incl., <mark>L. Kreidberg</mark>, <mark>S. Scheithauer</mark> |
|*Appeared on*| *2023-12-06*|
|*Comments*| *9 pages, 5 figures, submitted to MNRAS*|
|**Abstract**| The dark compact object at the centre of the Milky Way is well established to be a supermassive black hole with mass $M_{\bullet} \sim 4.3 \cdot 10^6 \, M_{\odot}$, but the nature of its environment is still under debate. In this work, we used astrometric and spectroscopic measurements of the motion of the star S2, one of the closest stars to the massive black hole, to determine an upper limit on an extended mass composed of a massive vector field around Sagittarius A*. For a vector with effective mass $10^{-19} \, \rm eV \lesssim m_s \lesssim 10^{-18} \, \rm eV$, our Markov Chain Monte Carlo analysis shows no evidence for such a cloud, placing an upper bound $M_{\rm cloud} \lesssim 0.1\% M_{\bullet}$ at $3\sigma$ confidence level. We show that dynamical friction exerted by the medium on S2 motion plays no role in the analysis performed in this and previous works, and can be neglected thus. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.02311-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.02311) | **Searching for the Highest-z Dual AGN in the Deepest Chandra Surveys**  |
|| B. Sandoval, et al. -- incl., <mark>M. Habouzit</mark> |
|*Appeared on*| *2023-12-06*|
|*Comments*| *19 pages, 5 figures, 2 tales. Submitted to ApJ*|
|**Abstract**| We present an analysis searching for dual AGN among 62 high-redshift ($2.5 < z < 3.5$) X-ray sources selected from publicly available deep Chandra fields. We aim to quantify the frequency of dual AGN in the high-redshift Universe, which holds implications for black hole merger timescales and low-frequency gravitational wave detection rates. We analyze each X-ray source using BAYMAX, an analysis tool that calculates the Bayes factor for whether a given archival Chandra AGN is more likely a single or dual point source. We find no strong evidence for dual AGN in any individual source in our sample. We then increase our sensitivity to search for dual AGN across the sample by comparing our measured distribution of Bayes factors to that expected from a sample composed entirely of single point sources, and again find no evidence for dual AGN in the observed sample distribution. Although our analysis utilizes one of the largest Chandra catalogs of high-$z$ X-ray point sources available to study, the findings remain limited by the modest number of sources observed at the highest spatial resolution with Chandra and the typical count rates of the detected sources. Our non-detection allows us to place an upper-limit on the X-ray dual AGN fraction between $2.5<z<3.5$ of 4.8\%. Expanding substantially on these results at X-ray wavelengths will require future surveys spanning larger sky areas and extending to fainter fluxes than has been possible with Chandra. We illustrate the potential of the AXIS mission concept in this regard. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2312.02653.md
    + _build/html/tmp_2312.02653/./orbital_elements_cloud+Schwarzschild.png
    + _build/html/tmp_2312.02653/./difference_wDF.png
    + _build/html/tmp_2312.02653/./orbital_elements_only_cloud.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\af}[1]{{\textcolor{red}{\sf{[Arianna: #1]}} }}$
$\newcommand{\pg}[1]{{\textcolor{magenta}{#1}}}$
$\newcommand{\sg}[1]{{\textcolor{cyan}{\sf{[Stefan: #1]}} }}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\af}[1]{{\textcolor{red}{\sf{[Arianna: #1]}} }}$
$\newcommand{\pg}[1]{{\textcolor{magenta}{#1}}}$
$\newcommand{\sg}[1]{{\textcolor{cyan}{\sf{[Stefan: #1]}} }}$
$\usepackage{graphicx}$
$\usepackage{amsmath}$
$\title[Vector clouds around SgrA^*]{Using the motion of S2 to constrain vector clouds around SgrA^*}$
$\author[GRAVITY Collaboration]{GRAVITY Collaboration \thanks{GRAVITY is developed in collaboration by$
$MPE,$
$LESIA of Paris Observatory / CNRS / Sorbonne Université / Univ. Paris Diderot$
$and IPAG of Université Grenoble Alpes / CNRS,$
$MPIA,$
$Univ. of  Cologne,$
$CENTRA - Centro de Astrofísica e Gravita\c cão, and ESO. Corresponding authors: A.~Foschi$
$(arianna.foschi@tecnico.ulisboa.pt) \& P.J.V.~Garcia (pgarcia@fe.up.pt)$
$}:$
$A.~Foschi^{1, 2},$
$R.~Abuter^{3},$
$K. Abd El Dayem^{4},$
$N.~Aimar^{4}, \newauthor$
$P.~Amaro Seoane^{6, 5, 7, 21},$
$A.~Amorim^{1, 8},$
$J.P.~Berger^{9},$
$H.~Bonnet^{3},$
$G.~Bourdarot^{5},$
$W.~Brandner^{10}, \newauthor$
$R.~Davies^{5},$
$P.T.~de~Zeeuw^{11},$
$D.~Defrère^{19},$
$J.~Dexter^{12},$
$A.~Drescher^{5},$
$A.~Eckart^{16, 18},$
$F.~Eisenhauer^{5}, \newauthor$
$N.M.~Förster~Schreiber^{5},$
$P.J.V.~Garcia^{1, 2},$
$R.~Genzel^{5, 13},$
$S.~Gillessen^{5},$
$T.~Gomes^{1, 2},$
$X.~Haubois^{14},\newauthor$
$G.~Hei{\ss}el^{4, 15},$
$Th.~Henning^{10},$
$L.~Jochum^{14},$
$L.~Jocou^{10},$
$A.~Kaufer^{14},$
$L.~Kreidberg^{10},$
$S.~Lacour^{4}, \newauthor$
$V.~Lapeyrère^{4},$
$J.-B.~Le~Bouquin^{9},$
$P.~Léna^{4},$
$D.~Lutz^{5},$
$F.~Mang^{5},$
$F.~Millour^{20},$
$T.~Ott^{5},$
$T.~Paumard^{4}, \newauthor$
$K.~Perraut^{9},$
$G.~Perrin^{4},$
$O.~Pfuhl^{3, 5},$
$S.~Rabien^{5},$
$D.C.~Ribeiro^{5},$
$M.~Sadun Bordoni^{5},$
$S.~Scheithauer^{10},  \newauthor$
$J.~Shangguan^{5},$
$T.~Shimizu^{5},$
$J.~Stadler^{5, 17},$
$C.~Straubmeier^{16},$
$E.~Sturm^{5},$
$M.~Subroweit^{16},$
$L.J.~Tacconi^{5},  \newauthor$
$F.~Vincent^{4},$
$S.~von~Fellenberg^{5, 18}$
$and J.~Woillez^{3}$
$\^{1}CENTRA - Centro de Astrofísica e$
$Gravita\c cão, IST, Universidade de Lisboa, 1049-001 Lisboa,$
$Portugal\^2Faculdade de Engenharia, Universidade do Porto, rua Dr. Roberto$
$Frias, 4200-465 Porto, Portugal\^3European Southern Observatory, Karl-Schwarzschild-Stra{\ss}e 2, 85748$
$Garching, Germany\^4LESIA, Observatoire de Paris, Université PSL, CNRS, Sorbonne Université, Université de Paris, 5 place Jules Janssen, 92195 Meudon, France\^5Max Planck Institute for extraterrestrial Physics,$
$Giessenbachstra{\ss}e~1, 85748 Garching, Germany\^{6}Universitat Politècnica de València, València, Spain \^{7} Kavli Institute for Astronomy and Astrophysics, Beijing, China \^{8}Universidade de Lisboa - Faculdade de Ci\^encias, Campo Grande,$
$1749-016 Lisboa, Portugal\^{9}Univ. Grenoble Alpes, CNRS, IPAG, 38000 Grenoble, France\^{10}Max Planck Institute for Astronomy, Königstuhl 17,$
$69117 Heidelberg, Germany\^{11}Leiden University, 2311EZ Leiden, The Netherlands\^{12}Department of Astrophysical \& Planetary Sciences, JILA, Duane Physics Bldg., 2000 Colorado Ave, University of Colorado, Boulder, CO 80309, USA\^{13}Departments of Physics and Astronomy, Le Conte Hall, University$
$of California, Berkeley, CA 94720, USA\^{14}European Southern Observatory, Casilla 19001, Santiago 19, Chile\^{15}Advanced Concepts Team, European Space Agency, TEC-SF, ESTEC, Keplerlaan 1, 2201, AZ Noordwijk, The Netherlands \^{16} 1^{\rm st} Institute of Physics, University of Cologne,$
$Zülpicher Stra{\ss}e 77, 50937 Cologne, Germany\^{17}Max Planck Institute for Astrophysics, Karl-Schwarzschild-Stra{\ss}e 1, D-85748$
$Garching, Germany\^{18}Max Planck Institute for Radio Astronomy, auf dem Hügel 69, D-53121 Bonn, Germany \^{19}Institute of Astronomy, KU Leuven, Celestijnenlaan 200D, 3001 Leuven, Belgium \^{20}Université C\^{o}te d'Azur, Observatoire de la  C\^{o}te d'Azur, CNRS, Lagrange, France \^{21} Higgs Centre for Theoretical Physics, Edinburgh, UK}$
$\date{Accepted XXX. Received YYY; in original form ZZZ}$
$\pubyear{2023}$
$\begin{document}$
$\label{firstpage}$
$\pagerange{\pageref{firstpage}--\pageref{lastpage}}$
$\maketitle$
$\begin{abstract}$
$The dark compact object at the centre of the Milky Way is well established to be a supermassive black hole with mass M_{\bullet} \sim 4.3 \cdot 10^6   M_{\odot}, but the nature of its environment is still under debate. In this work, we used astrometric and spectroscopic measurements of the motion of the star S2, one of the closest stars to the massive black hole, to determine an upper limit on an extended mass composed of a massive vector field around Sagittarius A*. For a vector with effective mass 10^{-19}   \rm eV \lesssim m_s \lesssim 10^{-18}   \rm eV, our Markov Chain Monte Carlo analysis shows no evidence for such a cloud, placing an upper bound M_{\rm cloud} \lesssim 0.1\% M_{\bullet} at 3\sigma confidence level.$
$We show that dynamical friction exerted by the medium on S2 motion plays no role in the analysis performed in this and previous works, and can be neglected thus.$
$\end{abstract}$
$\begin{keywords}$
$black holes physics -- dark matter -- gravitation -- celestial mechanics -- Galaxy: centre$
$\end{keywords}$
$\n\end{document}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$</div>



<div id="title">

# Using the motion of S2 to constrain vector clouds around SgrA$^*$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.02653-b31b1b.svg)](https://arxiv.org/abs/2312.02653)<mark>Appeared on: 2023-12-06</mark> -  _9 pages, 5 figures, submitted to MNRAS_

</div>
<div id="authors">

G. Collaboration, et al. -- incl., <mark>L. Kreidberg</mark>, <mark>S. Scheithauer</mark>

</div>
<div id="abstract">

**Abstract:** The dark compact object at the centre of the Milky Way is well established to be a supermassive black hole with mass $M_{\bullet} \sim 4.3 \cdot 10^6   M_{\odot}$ , but the nature of its environment is still under debate. In this work, we used astrometric and spectroscopic measurements of the motion of the star S2, one of the closest stars to the massive black hole, to determine an upper limit on an extended mass composed of a massive vector field around Sagittarius A*. For a vector with effective mass $10^{-19}   \rm eV \lesssim m_s \lesssim 10^{-18}   \rm eV$ , our Markov Chain Monte Carlo analysis shows no evidence for such a cloud, placing an upper bound $M_{\rm cloud} \lesssim 0.1\% M_{\bullet}$ at $3\sigma$ confidence level.We show that dynamical friction exerted by the medium on S2 motion plays no role in the analysis performed in this and previous works, and can be neglected thus.

</div>

<div id="div_fig1">

<img src="tmp_2312.02653/./orbital_elements_cloud+Schwarzschild.png" alt="Fig2" width="100%"/>

**Figure 2. -** Variation of the orbital elements $\Delta \mu^{a}$ over an entire orbit for different values of the coupling constant $\alpha$ when one includes the Schwarzschild precession in the equation for the osculating elements. Here $\Lambda = 10^{-3}$. The maximum variation is still found in $0.003 \lesssim \alpha \lesssim 0.03$. (*fig:var_elements_cloud_1PN*)

</div>
<div id="div_fig2">

<img src="tmp_2312.02653/./difference_wDF.png" alt="Fig5" width="100%"/>

**Figure 5. -** Absolute difference in DEC, R.A. and radial velocity between the case where dynamical friction is implemented in the supersonic case with $c_s = 10^{-3}$ and the case where no dynamical friction is present. In both cases $\Lambda = 10^{-3}$ and $\alpha = 0.015$. The difference is maximum around the periastron passages (red dashed lines) and minimum at the apoastron (black dotted line). Overall, they remain far below the current instrument threshold. (*fig:dynamical_friction*)

</div>
<div id="div_fig3">

<img src="tmp_2312.02653/./orbital_elements_only_cloud.png" alt="Fig1" width="100%"/>

**Figure 1. -** Variation of the orbital elements $\Delta \mu^{a}/\Lambda$ over an entire orbit for different values of the coupling constant $\alpha$ when only the vector cloud is present. The maximum variation in $\Delta \omega/\Lambda$ is roughly found in the range $0.003 \lesssim \alpha \lesssim 0.03$. (*fig:var_elements_cloud*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.02653"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

334  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
